In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import librosa

In [2]:
df = pd.read_csv('normalized_data.csv')

In [3]:
X = df.drop(columns=['Unnamed: 0', 'genre'])

In [7]:
y = df['genre']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
from sklearn import svm

# Train an SVM model
clf = svm.SVC(kernel='linear', C=0.2)
clf.fit(X_train, y_train)

# Evaluate the model on the test data
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.73


In [16]:
def normalize_volume(file_path):
    y, sr = librosa.load(file_path)
    y_norm = librosa.util.normalize(y, axis=0)
    return y_norm, sr

In [24]:
def extract_features(y_norm, sr):

    features = []

    # Tempo and beats
    tempo, beats = librosa.beat.beat_track(y=y_norm, sr=sr)
    beats_mean = beats.mean()
    beats_var = beats.var()
    features.extend((tempo, beats_mean, beats_var))

    # Zero crossings
    zero_crossings = librosa.zero_crossings(y=y_norm, pad=False)
    zero_crossings_mean = zero_crossings.mean()
    zero_crossings_var = zero_crossings.var()
    features.extend((zero_crossings_mean, zero_crossings_var))

    # Spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=y_norm, sr=sr)[0]
    spectral_centroids_mean = spectral_centroids.mean()
    spectral_centroids_var = spectral_centroids.var()
    features.extend((spectral_centroids_mean,spectral_centroids_var))

    # Specral Rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y_norm, sr=sr)[0]
    spectral_rolloff_mean = spectral_rolloff.mean()
    spectral_rolloff_var = spectral_rolloff.var()
    features.extend((spectral_rolloff_mean, spectral_rolloff_var))

    # MFCCs
    mfccs = librosa.feature.mfcc(y=y_norm, sr=sr, n_mfcc=40)
    for mfcc in mfccs:
        features.append(mfcc.mean())
        features.append(mfcc.var())

    return features

In [19]:
y_new, sr = normalize_volume('song_input.wav')

In [26]:
y_new_features = extract_features(y_new, sr)

In [28]:
y_new_features = pd.DataFrame(y_new_features)

In [30]:
y_new_features

,0
0,1.359992e+02
1,2.383744e+03
2,1.745494e+06
3,1.963157e-02
4,1.924617e-02
...,...
84,2.071960e+02
85,6.379077e-01
86,1.277039e+02
87,-2.132262e+00


In [31]:
clf.predict(y_new_features.T)

/home/rsmassey/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([8])